In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('movieReplicationSet.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097 entries, 0 to 1096
Columns: 477 entries, The Life of David Gale (2003) to Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
dtypes: float64(475), int64(2)
memory usage: 4.0 MB


In [34]:
# calculate column means
col_means = df.iloc[:, :1097].mean()

# calculate row means
row_means = df.iloc[:, :400].mean(axis=1)

# fill NaN values with 50:50 blend of column mean and row mean
df.iloc[:, :1097] = df.iloc[:, :1097].apply(lambda x: x.fillna((x.mean() + row_means) / 2), axis=1)
df.iloc[:, :400] = df.iloc[:, :400].apply(lambda x: x.fillna(col_means[x.name]), axis=0)

In [35]:
df.head()

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,2.0,2.0,4.0,3.0,3.0,3.0,2.0,3.0,2.0,3.0,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0.0,1.0
1,2.0,2.0,2.0,3.0,3.0,3.0,2.0,3.0,2.0,3.0,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0.0,0.0
2,2.0,2.0,3.0,3.0,3.0,3.0,2.0,3.0,2.0,3.0,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1.0,0.0
3,2.0,2.0,2.0,3.0,3.0,3.0,2.0,3.0,2.0,4.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0.0,1.0
4,2.0,2.0,4.0,3.0,0.0,3.0,0.0,1.0,2.0,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1.0,1.0


In [36]:
# Round all float values to the nearest integer
df = df.round()

In [37]:
from sklearn.linear_model import LinearRegression

In [ ]:
cod_values = {}
lr = LinearRegression()

for movie in range(400):
    max_cod = -1
    best_predictor = -1
    for predictor in range(400):
        if predictor != movie:
            X_pred = df.iloc[:, predictor].values.reshape(-1, 1)
            y_pred = df.iloc[:, movie].values
            lr.fit(X_pred, y_pred)
            cod = lr.score(X_pred, y_pred)
            if cod > max_cod:
                max_cod = cod
                best_predictor = predictor
    cod_values[movie] = max_cod

In [ ]:
# Report average COD
avg_cod = np.mean(list(cod_values.values()))
print('Average COD:', avg_cod)


In [ ]:
# Histogram of COD values
import matplotlib.pyplot as plt
plt.hist(list(cod_values.values()))
plt.title('Histogram of COD Values')
plt.xlabel('COD')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# create table of 10 easiest/hardest movies to predict
cod_df = pd.DataFrame({'Movie': list(cod_values.keys()), 'COD': list(cod_values.values())})
cod_df = cod_df.sort_values(by='COD', ascending=False)
best_predictors = []
worst_predictors = []
for i in range(10):
    movie = cod_df.iloc[i]['Movie']
    cod = cod_df.iloc[i]['COD']
    best_pred = df.columns[best_predictor]
    worst_pred = df.columns[cod_df.iloc[-(i+1)]['Movie']]
    best_predictors.append((movie, best_pred, cod))
    worst_predictors.append((movie, worst_pred, cod))
best_predictors_df = pd.DataFrame(best_predictors, columns=['Movie', 'Best Predictor', 'COD'])
worst_predictors_df = pd.DataFrame(worst_predictors, columns=['Movie', 'Worst Predictor', 'COD'])

# display results
print('Top 10 easiest movies to predict:')
display(best_predictors_df)

print('\nTop 10 hardest movies to predict:')
display(worst_predictors_df)


In [ ]:
print('10 easiest movies to predict:')
print(best_predictors_df)
print('\n10 hardest movies to predict:')
print(worst_predictors_df)
